In [1]:
import os, sys
import esutil
import healpy as hp
import numpy as np
import fitsio
sys.path.append('../')
from xd import *
from run_DMASS import *
from utils import *
import healpy as hp
from systematics import *
from cmass_modules import io
os.chdir('../../DMASS_XDGMM/code_py3/')
print(os.getcwd())
from xdgmm import XDGMM as XDGMM_Holoien
os.chdir('../../DMASSY3/code_py3/')
print(os.getcwd())

In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


/users/PCON0003/warner785/DMASS_XDGMM/code_py3
/users/PCON0003/warner785/DMASSY3/code_py3


In [2]:
#load in:
clf_130 = XDGMM_Holoien(filename='output/test/fitting/gauss_gold_bovyY3_130DET_unfixed.xdgmm')
clf_no130 = XDGMM_Holoien(filename='output/test/fitting/gauss_nogold_bovyY3_130DET_unfixed.xdgmm')

#load in:
clf_200 = XDGMM_Holoien(filename='output/test/fitting/gauss_gold_bovyY3_200DET_unfixed.xdgmm')
clf_no200 = XDGMM_Holoien(filename='output/test/fitting/gauss_nogold_bovyY3_200DET_unfixed.xdgmm')

#load in:
clf_300 = XDGMM_Holoien(filename='output/test/fitting/gauss_gold_bovyY3_300DET_unfixed.xdgmm')
clf_no300 = XDGMM_Holoien(filename='output/test/fitting/gauss_nogold_bovyY3_300DET_unfixed.xdgmm')

In [3]:
# call required functions from modules 

sys.path.append('code_py3/')
from cmass_modules import io
from utils import matchCatalogsbyPosition, hpHEALPixelToRaDec, HealPixifyCatalogs, spatialcheck
from xd import mixing_color, XD_fitting_X, assignCMASSProb, doVisualization_1d
from run_DMASS_Y3 import priorCut_test

# Calling Y3 GOLD v2.0 (training)
# All catalogs are in the 'input_path' directory 
# The 'SearchAndCallFits' function below loads all 
# catalogs in the directory including 'input_keyword' in its name
#input_path = '/n/des/lee.5922/data/gold_cat_Y3/STRIPE82/feae9705305d4430993687930f1cc3ad/'
input_path = '/fs/scratch/PCON0003/warner785/bwarner/'
# call only first 9 catalogs for a fast run.
# to call all catalogs in the directory, use 'Y3_GOLD' as input_keyword 
# but that will consume huge memory
input_keyword = 'Y3_GOLD_2_0_STRIPE82'  
# Columns to call
columns =  ['RA', 'DEC', 
            'HPIX_4096',     # Healpix in ring order, nside=4096
            'COADD_OBJECT_ID', 
            'SOF_CM_MAG_CORRECTED_G', # mag_[griz]
            'SOF_CM_MAG_CORRECTED_R',
            'SOF_CM_MAG_CORRECTED_I',
            'SOF_CM_MAG_CORRECTED_Z',
            'SOF_CM_MAG_ERR_G',       # mag error_[griz]
            'SOF_CM_MAG_ERR_R',
            'SOF_CM_MAG_ERR_I',
            'SOF_CM_MAG_ERR_Z', 'EXTENDED_CLASS_MASH_SOF', 'MAG_DETMODEL_G', 'MAG_DETMODEL_R', 'MAG_DETMODEL_I', 'MAG_DETMODEL_Z', 
            'MAGERR_DETMODEL_G', 'MAGERR_DETMODEL_R', 'MAGERR_DETMODEL_I', 'MAGERR_DETMODEL_Z',
            'MOF_CM_MAG_CORRECTED_G', 'MOF_CM_MAG_CORRECTED_R', 'MOF_CM_MAG_CORRECTED_I', 'MOF_CM_MAG_CORRECTED_Z',
            'MOF_CM_MAG_ERR_G', 'MOF_CM_MAG_ERR_R', 'MOF_CM_MAG_ERR_I', 'MOF_CM_MAG_ERR_Z']
gold_st82 = io.SearchAndCallFits(path = input_path, keyword = input_keyword, columns=columns)

# Color/Magnitude cuts to exclude extremely high or low mag/color sources.
# 16 < mag_riz < 24, 0 < (r-i) < 1.5, 0 < (g-r) < 2.5
# These galaxies are less likeliy to be a CMASS, therefore unnecessary. 
# We apply these cuts to reduce the sample size to speed up the codes
mask_magcut = priorCut_test(gold_st82)
gold_st82 = gold_st82[mask_magcut]
gold_st82 = gold_st82[(gold_st82['EXTENDED_CLASS_MASH_SOF'] == 3)]

def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/fs/scratch/PCON0003/warner785/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

print(gold_st82.size)    
gold_st82 = keepGoodRegion(gold_st82)
print(gold_st82.size)


Calling catalogs >>>
Y3_GOLD_2_0_STRIPE82_000125.fits
Y3_GOLD_2_0_STRIPE82_000136.fits
Y3_GOLD_2_0_STRIPE82_000040.fits
Y3_GOLD_2_0_STRIPE82_000015.fits
Y3_GOLD_2_0_STRIPE82_000086.fits
Y3_GOLD_2_0_STRIPE82_000047.fits
Y3_GOLD_2_0_STRIPE82_000108.fits
Y3_GOLD_2_0_STRIPE82_000079.fits
Y3_GOLD_2_0_STRIPE82_000076.fits
Y3_GOLD_2_0_STRIPE82_000057.fits
Y3_GOLD_2_0_STRIPE82_000111.fits
Y3_GOLD_2_0_STRIPE82_000084.fits
Y3_GOLD_2_0_STRIPE82_000088.fits
Y3_GOLD_2_0_STRIPE82_000049.fits
Y3_GOLD_2_0_STRIPE82_000063.fits
Y3_GOLD_2_0_STRIPE82_000070.fits
Y3_GOLD_2_0_STRIPE82_000077.fits
Y3_GOLD_2_0_STRIPE82_000042.fits
Y3_GOLD_2_0_STRIPE82_000118.fits
Y3_GOLD_2_0_STRIPE82_000115.fits
Y3_GOLD_2_0_STRIPE82_000028.fits
Y3_GOLD_2_0_STRIPE82_000140.fits
Y3_GOLD_2_0_STRIPE82_000060.fits
Y3_GOLD_2_0_STRIPE82_000089.fits
Y3_GOLD_2_0_STRIPE82_000143.fits
Y3_GOLD_2_0_STRIPE82_000012.fits
Y3_GOLD_2_0_STRIPE82_000001.fits
Y3_GOLD_2_0_STRIPE82_000045.fits
Y3_GOLD_2_0_STRIPE82_000137.fits
Y3_GOLD_2_0_STRIPE82_

In [4]:
#this CMASS used for Y1, and Y3 now:

import fitsio
cmass = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
train_sample = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
#train_sample = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')   
train_sample = keepGoodRegion(train_sample)

#fitsio.write( output_dir+'/cmass_in_st82.fits', train_sample)

print('num of train_sample after des veto', train_sample.size)

total num of train 258884

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 61031


In [5]:
#(Validation) 130

mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)
mask = mask & (gold_st82['DEC']>-2) & (gold_st82['DEC']<2)
gold_st82_train = gold_st82[mask]

print(gold_st82_train.size)

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_train['RA'], \
                                         gold_st82_train['DEC'],2./3600, maxmatch=1)
cmass_mask_train = np.zeros(gold_st82_train.size, dtype=bool)
cmass_mask_train[mg2] = 1
clean_cmass_data_des_train, nocmass_train = gold_st82_train[cmass_mask_train], gold_st82_train[~cmass_mask_train]

cmass_fraction_130 = clean_cmass_data_des_train.size *1./gold_st82_train.size
print(cmass_fraction_130)

mag = ['MAG_DETMODEL', 'MAG_DETMODEL']
err = ['MAGERR_DETMODEL', 'MAGERR_DETMODEL']
filter = ['G', 'R', 'I']

mask4 =(gold_st82['RA']>18)&(gold_st82['RA']<43)
mask4 = mask4 & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_v4 = gold_st82[mask4]

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_v4['RA'], \
                                         gold_st82_v4['DEC'],2./3600, maxmatch=1)
cmass_mask4 = np.zeros(gold_st82_v4.size, dtype=bool)
cmass_mask4[mg2] = 1
clean_cmass_data_des_v4, nocmass_v4 = gold_st82_v4[cmass_mask4], gold_st82_v4[~cmass_mask4]

clean_dmass_prob1 = assignCMASSProb(clean_cmass_data_des_v4, clf_130, clf_no130, 
                     cmass_fraction =cmass_fraction_130
, 
                     mag=mag, err=err, filter=filter ) 
dmass_st82_real1 = clean_dmass_prob1

dmass_st82_val1 = assignCMASSProb(gold_st82_v4, clf_130, clf_no130, 
                     cmass_fraction = cmass_fraction_130
, 
                     mag=mag, err=err, filter=filter )


pbin = np.linspace(0, 1, 21)
pcenter = pbin[:-1] + 0.025
N, _ = np.histogram(dmass_st82_real1['CMASS_PROB'], bins=pbin)
print('pbin =', pbin)
import matplotlib.pyplot as plt
#pbin, frac, fracerr = np.genfromtxt('../paper_figure/model_probability.txt', unpack=True)

M, _ = np.histogram(dmass_st82_val1['CMASS_PROB'], bins=pbin)

frac = N/M
print('frac = ', frac)
fracerr = 1/np.sqrt(N) #1 / sqrt(number of true cmass galaxies in each bin)
#fracerr = (N/M)*np.sqrt((1/N)-(1/M))

x1=frac
x2=pcenter
err=fracerr
diff2 = (x1-x2)**2 
diff2_reduced = sum(diff2)/diff2.size
print("diff2_reduced: ",diff2_reduced)
print("size: ",diff2.size)
print("sum of diff2: ",sum(diff2))

fig, ax = plt.subplots()
ax.errorbar( pcenter, frac, yerr=fracerr)
ax.plot([0,1], [0,1], ls='--', color='grey')
#ax.legend(chi2_reduced)
plt.title('Validation Region Check 130 sqdeg')
fig.savefig('DET_Val_130_check.pdf')

#describe plot: ***used for model and validation sample-- what is done to create plot

1192126
0.01106678320915742
calculate loglikelihood gaussian with multiprocessing module
multiprocessing 50 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)
/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 58 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 62 %

...

multiprocessing 92 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 96 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 100 % 
pbin = [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
frac =  [0.00119592 0.08839616 0.14248429 0.1818317  0.23367084 0.28529194
 0.35047951 0.37857618 0.43715847 0.48393417 0.53333333 0.58050847
 0.63779528 0.68059701 0.73067206 0.78919492 0.83100108 0.87650775
 0.90188249 0.91477717]
diff2_reduced:  0.0003448604886329987
size:  20
sum of diff2:  0.006897209772659974


In [7]:
#training region check

#mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)
#mask = mask & (gold_st82['DEC']>-2) & (gold_st82['DEC']<2)
#gold_st82_train = gold_st82[mask]

mag = ['MAG_DETMODEL', 'MAG_DETMODEL']
err = ['MAGERR_DETMODEL', 'MAGERR_DETMODEL']
filter = ['G', 'R', 'I']

mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)|(gold_st82['RA']<7)
mask = mask & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_train = gold_st82[mask]

print(gold_st82_train.size)

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_train['RA'], \
                                         gold_st82_train['DEC'],2./3600, maxmatch=1)
cmass_mask_train = np.zeros(gold_st82_train.size, dtype=bool)
cmass_mask_train[mg2] = 1
clean_cmass_data_des_train, nocmass_train = gold_st82_train[cmass_mask_train], gold_st82_train[~cmass_mask_train]

cmass_fraction_200 = clean_cmass_data_des_train.size *1./gold_st82_train.size
print(cmass_fraction_200)

#clean_cmass_data_des = clean_cmass_data_des_train

clean_dmass_prob_train = assignCMASSProb(clean_cmass_data_des_train, clf_200, clf_no200, 
                     cmass_fraction =cmass_fraction_200
, 
                     mag=mag, err=err, filter=filter )


dmass_st82_real_train = clean_dmass_prob_train # actual CMASS galaxies in train set
dmass_st82_val_train = assignCMASSProb(gold_st82_train, clf_200, clf_no200, 
                     cmass_fraction = cmass_fraction_200
, 
                     mag=mag, err=err, filter=filter )

pbin = np.linspace(0, 1, 21)
pcenter = pbin[:-1] + 0.025
N, _ = np.histogram(dmass_st82_real_train['CMASS_PROB'], bins=pbin)

import matplotlib.pyplot as plt
#pbin, frac, fracerr = np.genfromtxt('../paper_figure/model_probability.txt', unpack=True)

M, _ = np.histogram(dmass_st82_val_train['CMASS_PROB'], bins=pbin)

frac = N/M
fracerr = 1/np.sqrt(N) #1 / sqrt(number of true cmass galaxies in each bin)

x1=frac
x2=pcenter
err=fracerr
chi2 = (x1-x2)**2 / err **2 
chi2_reduced = sum(chi2)/chi2.size
print("chi2_reduced: ",chi2_reduced)
print("size: ",chi2.size)
print("sum of chi2: ",sum(chi2))

fig, ax = plt.subplots()
ax.errorbar( pcenter, frac, yerr=fracerr, label = str(sum(chi2)))
ax.plot([0,1], [0,1], ls='--', color='grey')
plt.title('Training Region Check 100%')
fig.savefig('DET200_train.pdf')

1859266
0.011108684825086888
calculate loglikelihood gaussian with multiprocessing module
multiprocessing 50 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 54 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 58 %

...

multiprocessing 92 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 96 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 100 % 
chi2_reduced:  0.5692805369344482
size:  20
sum of chi2:  11.385610738688964


In [5]:
#(Validation) 200

mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)|(gold_st82['RA']<7)
mask = mask & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_train = gold_st82[mask]

print(gold_st82_train.size)

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_train['RA'], \
                                         gold_st82_train['DEC'],2./3600, maxmatch=1)
cmass_mask_train = np.zeros(gold_st82_train.size, dtype=bool)
cmass_mask_train[mg2] = 1
clean_cmass_data_des_train, nocmass_train = gold_st82_train[cmass_mask_train], gold_st82_train[~cmass_mask_train]

cmass_fraction_200 = clean_cmass_data_des_train.size *1./gold_st82_train.size
print(cmass_fraction_200)

mag = ['MAG_DETMODEL', 'MAG_DETMODEL']
err = ['MAGERR_DETMODEL', 'MAGERR_DETMODEL']
filter = ['G', 'R', 'I']

mask4 =(gold_st82['RA']>18)&(gold_st82['RA']<43)
mask4 = mask4 & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_v4 = gold_st82[mask4]

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_v4['RA'], \
                                         gold_st82_v4['DEC'],2./3600, maxmatch=1)
cmass_mask4 = np.zeros(gold_st82_v4.size, dtype=bool)
cmass_mask4[mg2] = 1
clean_cmass_data_des_v4, nocmass_v4 = gold_st82_v4[cmass_mask4], gold_st82_v4[~cmass_mask4]

clean_dmass_prob2 = assignCMASSProb(clean_cmass_data_des_v4, clf_200, clf_no200, 
                     cmass_fraction =cmass_fraction_200
, 
                     mag=mag, err=err, filter=filter ) 
dmass_st82_real2 = clean_dmass_prob2
# healpix number apart of dmass val catalog? -- use gold column of hpix 
#different way:
#
dmass_st82_val2 = assignCMASSProb(gold_st82_v4, clf_200, clf_no200, 
                     cmass_fraction = cmass_fraction_200
, 
                     mag=mag, err=err, filter=filter )


pbin = np.linspace(0, 1, 21)
pcenter = pbin[:-1] + 0.025
N, _ = np.histogram(dmass_st82_real2['CMASS_PROB'], bins=pbin)
print('pbin =', pbin)
import matplotlib.pyplot as plt
#pbin, frac, fracerr = np.genfromtxt('../paper_figure/model_probability.txt', unpack=True)

M, _ = np.histogram(dmass_st82_val2['CMASS_PROB'], bins=pbin)

frac = N/M
print('frac = ', frac)
fracerr = 1/np.sqrt(N) #1 / sqrt(number of true cmass galaxies in each bin)
#fracerr = (N/M)*np.sqrt((1/N)-(1/M))

x1=frac
x2=pcenter
err=fracerr
diff2 = (x1-x2)**2 
diff2_reduced = sum(diff2)/diff2.size
print("diff2_reduced: ",diff2_reduced)
print("size: ",diff2.size)
print("sum of diff2: ",sum(diff2))

fig, ax = plt.subplots()
ax.errorbar( pcenter, frac, yerr=fracerr)
ax.plot([0,1], [0,1], ls='--', color='grey')
#ax.legend(chi2_reduced)
plt.title('Validation Region Check 200 sqdeg')
fig.savefig('DET_Val_200_check.pdf')

#describe plot: ***used for model and validation sample-- what is done to create plot

1859266
0.011108684825086888
calculate loglikelihood gaussian with multiprocessing module
multiprocessing 50 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)
/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 58 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 62 %

...

multiprocessing 92 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 96 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 100 % 
pbin = [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
frac =  [0.00103102 0.07772113 0.13092755 0.17614035 0.22640061 0.2735959
 0.32290143 0.37896762 0.42223762 0.48533951 0.52835052 0.57427938
 0.62295082 0.67628362 0.73654244 0.7994695  0.83350785 0.8693522
 0.89326146 0.91966173]
diff2_reduced:  0.0002840242207636532
size:  20
sum of diff2:  0.005680484415273064


In [6]:
#(Validation) 200

mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)|(gold_st82['RA']<7)
mask = mask & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_train = gold_st82[mask]

print(gold_st82_train.size)

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_train['RA'], \
                                         gold_st82_train['DEC'],2./3600, maxmatch=1)
cmass_mask_train = np.zeros(gold_st82_train.size, dtype=bool)
cmass_mask_train[mg2] = 1
clean_cmass_data_des_train, nocmass_train = gold_st82_train[cmass_mask_train], gold_st82_train[~cmass_mask_train]

cmass_fraction_200 = clean_cmass_data_des_train.size *1./gold_st82_train.size
print(cmass_fraction_200)

mag = ['MAG_DETMODEL', 'MAG_DETMODEL']
err = ['MAGERR_DETMODEL', 'MAGERR_DETMODEL']
filter = ['G', 'R', 'I']

mask4 =(gold_st82['RA']>18)&(gold_st82['RA']<43)
mask4 = mask4 & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_v4 = gold_st82[mask4]

print(gold_st82_v4.shape)

1859266
0.011108684825086888
(2741190,)


In [8]:
print(gold_st82_v4['HPIX_4096'])

[71771941 71771626 71773248 ... 72968986 72905874 72968681]


In [5]:
#(Validation) 300

mask = (gold_st82['RA']>310) & (gold_st82['RA']<360)|(gold_st82['RA']<18)
mask = mask & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_train = gold_st82[mask]

print(gold_st82_train.size)

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_train['RA'], \
                                         gold_st82_train['DEC'],2./3600, maxmatch=1)
cmass_mask_train = np.zeros(gold_st82_train.size, dtype=bool)
cmass_mask_train[mg2] = 1
clean_cmass_data_des_train, nocmass_train = gold_st82_train[cmass_mask_train], gold_st82_train[~cmass_mask_train]

clean_cmass_data_des = clean_cmass_data_des_train

cmass_fraction_300 = clean_cmass_data_des_train.size *1./gold_st82_train.size
print(cmass_fraction_300)

mag = ['MAG_DETMODEL', 'MAG_DETMODEL']
err = ['MAGERR_DETMODEL', 'MAGERR_DETMODEL']
filter = ['G', 'R', 'I']

mask4 =(gold_st82['RA']>18)&(gold_st82['RA']<43)
mask4 = mask4 & (gold_st82['DEC']>-10) & (gold_st82['DEC']<10)
gold_st82_v4 = gold_st82[mask4]

mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82_v4['RA'], \
                                         gold_st82_v4['DEC'],2./3600, maxmatch=1)
cmass_mask4 = np.zeros(gold_st82_v4.size, dtype=bool)
cmass_mask4[mg2] = 1
clean_cmass_data_des_v4, nocmass_v4 = gold_st82_v4[cmass_mask4], gold_st82_v4[~cmass_mask4]

clean_dmass_prob3 = assignCMASSProb(clean_cmass_data_des_v4, clf_300, clf_no300, 
                     cmass_fraction =cmass_fraction_300
, 
                     mag=mag, err=err, filter=filter ) 
dmass_st82_real3 = clean_dmass_prob3

dmass_st82_val3 = assignCMASSProb(gold_st82_v4, clf_300, clf_no300, 
                     cmass_fraction = cmass_fraction_300
, 
                     mag=mag, err=err, filter=filter )


pbin = np.linspace(0, 1, 21)
pcenter = pbin[:-1] + 0.025
N, _ = np.histogram(dmass_st82_real3['CMASS_PROB'], bins=pbin)
print('pbin =', pbin)
import matplotlib.pyplot as plt
#pbin, frac, fracerr = np.genfromtxt('../paper_figure/model_probability.txt', unpack=True)

M, _ = np.histogram(dmass_st82_val3['CMASS_PROB'], bins=pbin)

frac = N/M
print('frac = ', frac)
fracerr = 1/np.sqrt(N) #1 / sqrt(number of true cmass galaxies in each bin)
#fracerr = (N/M)*np.sqrt((1/N)-(1/M))

x1=frac
x2=pcenter
err=fracerr
diff2 = (x1-x2)**2 
diff2_reduced = sum(diff2)/diff2.size
print("diff2_reduced: ",diff2_reduced)
print("size: ",diff2.size)
print("sum of diff2: ",sum(diff2))

fig, ax = plt.subplots()
ax.errorbar( pcenter, frac, yerr=fracerr)
ax.plot([0,1], [0,1], ls='--', color='grey')
#ax.legend(chi2_reduced)
plt.title('Validation Region Check 300 sqdeg')
fig.savefig('DET_Val_300_check.pdf')

#describe plot: ***used for model and validation sample-- what is done to create plot

2815886
0.01060128144392209
calculate loglikelihood gaussian with multiprocessing module
multiprocessing 50 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)
/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 54 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 58 %

...

multiprocessing 92 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 96 %

/users/PCON0003/warner785/.conda/envs/DES_env/lib/python3.7/site-packages/astroML/density_estimation/xdeconv.py:130: RuntimeWarning: divide by zero encountered in log
  return log_multivariate_gaussian(X, self.mu, T) + np.log(self.alpha)


multiprocessing 100 % 
pbin = [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
frac =  [0.00101152 0.07915858 0.13110868 0.17387061 0.2302606  0.27480916
 0.3338142  0.37451613 0.42683351 0.50055617 0.52489796 0.5682778
 0.62797203 0.68578431 0.76253165 0.77379784 0.84244533 0.87190527
 0.88719684 0.92095915]
diff2_reduced:  0.00038186290679988666
size:  20
sum of diff2:  0.007637258135997733


In [6]:
# save the samples. 
outdir = '../output/test/train_cat/y3/'
os.makedirs(outdir, exist_ok=True)
esutil.io.write( outdir+'dmass_st82_DET200.fits', dmass_st82_val2, overwrite=True)

In [8]:
# save the samples. 
outdir = '../output/test/train_cat/y3/'
os.makedirs(outdir, exist_ok=True)
esutil.io.write( outdir+'dmass_st82_DET200train.fits', dmass_st82_val_train, overwrite=True)